In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
epil_df = pd.read_csv('datasets/epil.csv', sep=',')
epil_df.head()

,Unnamed: 0,y,trt,base,age,V4,subject,period,lbase,lage
0,1,5,placebo,11,31,0,1,1,-0.756354,0.114204
1,2,3,placebo,11,31,0,1,2,-0.756354,0.114204
2,3,3,placebo,11,31,0,1,3,-0.756354,0.114204
3,4,3,placebo,11,31,1,1,4,-0.756354,0.114204
4,5,3,placebo,11,30,0,2,1,-0.756354,0.081414


In [4]:
seizure_df = pd.read_csv('datasets/seizure (1).csv', sep=',')
seizure_df.head()

,Unnamed: 0,y1,y2,y3,y4,trt,base,age
0,1,5,3,3,3,0,11,31
1,2,3,5,3,3,0,11,30
2,3,2,4,0,5,0,6,25
3,4,4,4,1,4,0,8,36
4,5,7,18,9,21,0,66,22
